# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacaweather_df=pd.read_csv (r"output_data/cities.csv")
print(vacaweather_df)

     Unnamed: 0        City      Lat       Lng  Max Temp(f)  Humidity  \
0             0      Hobart -42.8794  147.3294        49.86        76   
1             1   Taoudenni  22.6783   -3.9836       111.88        11   
2             2  Codrington -38.2667  141.9667        44.98        93   
3             3      Cairns -16.9167  145.7667        69.75        88   
4             4    Khatanga  71.9667  102.5000        44.78        73   
..          ...         ...      ...       ...          ...       ...   
262         262       Jilib   0.4883   42.7854        74.35        82   
263         263       Basco  20.4487  121.9702        83.32        79   
264         264        Hofn  64.2539  -15.2082        54.63        56   
265         265    Katsuura  35.1333  140.3000        79.99        90   
266         266      Safaga  26.7292   33.9365        89.74        38   

     Cloudiness  Wind Speed Country        Date  
0            98       12.66      AU  1659287428  
1            61        

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(g_key)

In [4]:
locations = vacaweather_df[["Lat","Lng"]]
humidity = vacaweather_df["Humidity"]

In [5]:
locations

,Lat,Lng
0,-42.8794,147.3294
1,22.6783,-3.9836
2,-38.2667,141.9667
3,-16.9167,145.7667
4,71.9667,102.5000
...,...,...
262,0.4883,42.7854
263,20.4487,121.9702
264,64.2539,-15.2082
265,35.1333,140.3000


In [6]:
fig=gmaps.figure()
# df = gmaps.datasets.vacaweather_df()
heat_layer=gmaps.heatmap_layer(
        locations, weights=humidity, 
        dissipating=False, 
        max_intensity=100, 
        point_radius=1
        )
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
perftemp_df = vacaweather_df.loc[(vacaweather_df["Max Temp(f)"]>70) & (vacaweather_df["Max Temp(f)"]<80)]
perftemp_df.count()
perfwind_df = perftemp_df.loc[(vacaweather_df["Wind Speed"]< 10)]
perfwind_df.count()
perfvacaweather_df = perfwind_df[(vacaweather_df["Cloudiness"]<25)]
perfvacaweather_df




/var/folders/v0/fgqm4drj0fv_rywhmh177t240000gp/T/ipykernel_12007/3635688232.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  perfvacaweather_df = perfwind_df[(vacaweather_df["Cloudiness"]<25)]


Unnamed: 0     12
City           12
Lat            12
Lng            12
Max Temp(f)    12
Humidity       12
Cloudiness     12
Wind Speed     12
Country        12
Date           12
dtype: int64

In [35]:
perfvacaweather_df


,Unnamed: 0,City,Lat,Lng,Max Temp(f),Humidity,Cloudiness,Wind Speed,Country,Date
18,18,Córdoba,-31.4135,-64.1811,77.41,23,0,5.75,AR,1659287714
56,56,Laguna,38.4210,-121.4238,76.32,71,0,6.91,US,1659287729
64,64,Avarua,-21.2078,-159.7750,73.45,68,20,6.91,CK,1659287732
67,67,Lompoc,34.6391,-120.4579,79.02,77,0,4.00,US,1659287733
69,69,Jamestown,42.0970,-79.2353,79.02,53,0,6.91,US,1659287520
82,82,Castro,-24.7911,-50.0119,70.21,38,0,6.06,BR,1659287739
119,119,Aljezur,37.3191,-8.8033,79.20,55,4,7.14,PT,1659287751
123,123,Nanzhou,29.3596,112.4024,78.55,71,0,5.01,CN,1659287752
185,185,Mimongo,-1.6195,11.6068,78.48,53,21,4.14,GA,1659287783
227,227,Yicheng,31.3705,119.8703,79.16,90,5,4.99,CN,1659287798


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
hotel_df = perfvacaweather_df.insert(0,"Hotel Name", [12],allow_duplicates=False)
print(hotel_df)

ValueError: Length of values (1) does not match length of index (12)

In [62]:
# geocoordinates


target_type="hotel"
Lat= perfvacaweather_df("Lat")
Lng=perfvacaweather_df("Lng")

radius = 5000
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params= {
        "location": f"{Lat},{Lng}",
        "radius": radius,
        "types": "Hotel",
        "key": g_key,
}

for hotel in perfvacaweather_df["Hotels"].iterrows:
        locations= location
        params["keyword"] = Hotel
        Print(f"Retrieving Results for Index{index}: {hotel}.")
        response = requests.get(base_url, params=params).json

        response=requests.get(base_url,params)
        results = response["Hotels"]
        try:
            print(f"Closest Hotel is{results[0]")
            
        except (KeyError, IndexError):
            print("Missing Hotel in Area")
        print("------------------")


    

SyntaxError: f-string: expecting '}' (2690294621.py, line 26)

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
